In [2]:
import os
from datetime import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# 1. Setup the "Switcher" (Same as before)
def get_llm():
    llm_type = os.getenv("LLM_TYPE", "ollama")
    if llm_type == "groq":
        return ChatGroq(model="llama-3.1-8b-instant", temperature=0)
    else:
        return ChatOllama(model="llama3.1", temperature=0)

# 2. Define the Tools
# The docstring ("""...""") is CRITICAL. The AI reads this to know WHEN to use the tool.

@tool
def get_current_time():
    """Returns the current time in H:M:S format."""
    now = datetime.now()
    return now.strftime("%H:%M:%S")

@tool
def multiply_numbers(a: float, b: float):
    """Multiplies two numbers together. Use this for math."""
    return a * b

# 3. Bind tools to the LLM
llm = get_llm()
tools = [get_current_time, multiply_numbers]

# This tells the LLM: "Here are the tools you possess."
llm_with_tools = llm.bind_tools(tools)

# 4. Test it!
query = "What time is it right now?"

# print(f"User: {query}")
result = llm_with_tools.invoke(query)

# 5. Inspect the output
print("\n--- AI Raw Output ---")
print(result.tool_calls)

# 1. Ask the question
# query = "What time is it right now?"
print(f"User: {query}")

# 2. Get the AI's "Plan" (The Result)
# result = llm_with_tools.invoke(query)

# 3. Check if the AI wants to use a tool
if result.tool_calls:
    # Get the first tool request
    tool_request = result.tool_calls[0] 
    tool_name = tool_request["name"]
    tool_args = tool_request["args"]
    
    print(f"\n🤖 AI Request: I want to call function '{tool_name}'")
    
    # 4. MANUALLY run the tool (Acting as the computer)
    if tool_name == "get_current_time":
        # We call the actual python function here
        answer = get_current_time.invoke(tool_args)
        print(f"⚡ System Output: {answer}")
        
else:
    print(f"AI: {result.content}")



--- AI Raw Output ---
[{'name': 'get_current_time', 'args': {}, 'id': 'dc37msf1f', 'type': 'tool_call'}]
User: What time is it right now?

🤖 AI Request: I want to call function 'get_current_time'
⚡ System Output: 12:54:09
